In [51]:
#Import required libraries
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy

In [52]:
#Import project specific functions
#sys.path.append('..')
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [53]:
#Import shared functions
sys.path.append('..')
#sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *
from IPM_Shared_Code_public.Python.google_functions import *

### Use the config file to setup connections

In [54]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [55]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [56]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('ambassador_cw')

In [57]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [58]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [59]:
sql = '''select *
         from (select *,
                      row_number() over(partition by site_desc, park_borough order by site_desc) as n
               from crowdsdb.dbo.tbl_ref_park_sites
               where active = 1) as t
         where n = 1'''

In [60]:
site_ref = (pd.read_sql(con = engine, sql = sql)
            .rename(columns = {'park_borough': 'borough'}))[['site_id', 'site_desc', 'borough']]

### Read the current data from SQL

In [61]:
sql = 'select * from crowdsdb.dbo.tbl_cw_ambassador'

In [62]:
ambass_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['ambassador_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [63]:
if ambass_sql.shape()[0] > 0:
    format_datetime(ambass_sql, 'encounter_timestamp')
    format_datetime(ambass_sql, 'encounter_datetime')

In [64]:
sql_cols = list(ambass_sql.columns.values)

In [65]:
float_cols = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']
for c in float_cols:
    ambass_sql[c] = ambass_sql[c].astype(float)

In [66]:
hash_rows(ambass_sql, exclude_cols = ['site_id', 'encounter_timestamp', 'encounter_datetime'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [67]:
ambass = (read_google_sheet(cred_file, '_COMBINED Inter-Agency Parks Social Distancing Education Ambassador Report (Responses)', 
                            'RESPONSES', drop_empty_cols = False, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [68]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
1,6/30/2020 15:08:51,NaN,Queensbridge Park,NaN,Law Department,Raquel,Cromartie,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
2,6/30/2020 14:40:28,6/30/2020 13:08:00,Prospect Park,NaN,OATH,Alex,Kolodka,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
3,6/30/2020 14:17:34,6/30/2020 14:16:00,Bowne Park,Playground,OATH,Samad,Newaz,Jaruez,Davis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
4,6/30/2020 14:03:59,NaN,Owl's Head Park,NaN,DDC,Joseph,Stankus,NaN,NaN,NaN,...,Open field/multi-purpose play area,NaN,Group of 10 with no masks . Provided them Mask...,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
5,6/30/2020 13:54:55,6/30/2020 13:54:00,Prospect Park,NaN,OATH,Alan,Sookra,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn


In [69]:
yesno = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']

In [70]:
yesno_cols(ambass, yesno)

In [71]:
format_datetime(ambass, 'encounter_timestamp')
format_datetime(ambass, 'encounter_datetime')

In [72]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_comments,closed_amenity,closed_patroncount,closed_comments,borough,sd_pdcontact,closed_approach,closed_outcome,closed_pdcontact
1,06-30-2020 15:08:51,NaT,Queensbridge Park,NaN,Law Department,Raquel,Cromartie,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Queens,NaN,NaN,NaN,NaN
2,06-30-2020 14:40:28,06-30-2020 13:08:00,Prospect Park,NaN,OATH,Alex,Kolodka,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Brooklyn,NaN,NaN,NaN,NaN
3,06-30-2020 14:17:34,06-30-2020 14:16:00,Bowne Park,Playground,OATH,Samad,Newaz,Jaruez,Davis,NaN,...,NaN,NaN,NaN,NaN,NaN,Queens,NaN,NaN,NaN,NaN
4,06-30-2020 14:03:59,NaT,Owl's Head Park,NaN,DDC,Joseph,Stankus,NaN,NaN,NaN,...,Open field/multi-purpose play area,Group of 10 with no masks . Provided them Mask...,NaN,NaN,NaN,Brooklyn,NaN,NaN,NaN,NaN
5,06-30-2020 13:54:55,06-30-2020 13:54:00,Prospect Park,NaN,OATH,Alan,Sookra,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Brooklyn,NaN,NaN,NaN,NaN


In [73]:
#Remove rows with no timestamp because these rows have no data
ambass = ambass[ambass['encounter_timestamp'].notnull()]

In [74]:
ambass = ambass.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [53]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,06-15-2020 08:45:10,06-14-2020 18:33:00,Q015,NaN,DEP,Jesus,La Torre,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1,06-15-2020 07:45:42,06-13-2020 18:00:00,M088,9th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
2,06-15-2020 07:40:03,06-13-2020 17:26:00,M088,8th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
3,06-15-2020 07:35:59,06-13-2020 16:56:00,M088,7th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
4,06-15-2020 07:33:49,06-13-2020 16:07:00,M088,7th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [54]:
hash_rows(ambass, exclude_cols = ['site_id', 'encounter_timestamp', 'encounter_datetime'], hash_name = 'row_hash')

### Find the deltas based on the row hashes

In [55]:
ambass_deltas = (check_deltas(new_df = ambass, old_df = ambass_sql, on = ['encounter_timestamp', 'site_id', 'encounter_datetime'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [56]:
ambass_deltas.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough,dml_verb
0,06-15-2020 08:45:10,06-14-2020 18:33:00,Q015,NaN,DEP,Jesus,La Torre,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I
1,06-15-2020 07:45:42,06-13-2020 18:00:00,M088,9th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
2,06-15-2020 07:40:03,06-13-2020 17:26:00,M088,8th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
3,06-15-2020 07:35:59,06-13-2020 16:56:00,M088,7th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
4,06-15-2020 07:33:49,06-13-2020 16:07:00,M088,7th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I


In [57]:
ambass_inserts = ambass_deltas[ambass_deltas['dml_verb'] == 'I'][sql_cols]

In [58]:
ambass_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,06-15-2020 08:45:10,06-14-2020 18:33:00,Q015,NaN,DEP,Jesus,La Torre,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1,06-15-2020 07:45:42,06-13-2020 18:00:00,M088,9th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
2,06-15-2020 07:40:03,06-13-2020 17:26:00,M088,8th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
3,06-15-2020 07:35:59,06-13-2020 16:56:00,M088,7th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
4,06-15-2020 07:33:49,06-13-2020 16:07:00,M088,7th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [59]:
ambass_inserts.to_sql('tbl_cw_ambassador', engine, index = False, if_exists = 'append')

In [60]:
ambass_updates = ambass_deltas[ambass_deltas['dml_verb'] == 'U'][sql_cols]

In [61]:
ambass_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
1258,06-09-2020 11:34:18,05-30-2020 16:30:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1259,06-09-2020 11:34:18,05-30-2020 16:30:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1266,06-09-2020 11:34:14,05-28-2020 16:25:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1267,06-09-2020 11:34:14,05-28-2020 16:25:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1270,06-09-2020 11:34:13,05-27-2020 14:30:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [62]:
sql_update(ambass_updates, 'tbl_cw_ambassador', engine, ['encounter_timestamp', 'encounter_datetime', 'site_id'])